In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [5]:


labels = []
#read text file
with open('labels.txt') as f:
    labels = f.readlines()
    labels = [x.strip() for x in labels]


In [24]:
mobile_net =  tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

17225924/17225924 [==============================] - 1s 0us/step


In [6]:
original_model = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

91884032/91884032 [==============================] - 6s 0us/step


In [7]:
original_model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [26]:
mobile_net.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [27]:
#convert to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(mobile_net)
tflite_model = converter.convert()
#open file
open("converted_model_mobileNet.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\emilk\AppData\Local\Temp\tmpdqje8oc5\assets


INFO:tensorflow:Assets written to: C:\Users\emilk\AppData\Local\Temp\tmpdqje8oc5\assets


16904124

In [33]:
# run the model in inference mode
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()


# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(output_details)

# read an image and resize it
img = tf.keras.preprocessing.image.load_img(r"C:\Users\oskar\OneDrive - Jonkoping University\Dokument\SOTA\imagenet-mini\val\n02102177\ILSVRC2012_val_00016315.JPEG", target_size=(299, 299))

img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array

interpreter.set_tensor(input_details[0]['index'], [img_array])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
# get the top 5 predictions
top_5 = np.argsort(output_data[0])[-5:]
top_5
# replace the labels with labels from the text file
for i in top_5:
    print(labels[i])

[{'name': 'StatefulPartitionedCall:0', 'index': 227, 'shape': array([   1, 1000]), 'shape_signature': array([  -1, 1000]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
clumber, clumber spaniel,
Brittany spaniel,
English springer, English springer spaniel,
cocker spaniel, English cocker spaniel, cocker,
Welsh springer spaniel,


True

In [33]:
import cv2
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 299)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 299)

interpreter = tf.lite.Interpreter(model_path="converted_model_mobileNet.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


def predict_video(cap, interpreter, input_details, output_details, labels):

    while True:
        #count = 0
        success, img = cap.read()
        #reszie the image
        img = cv2.resize(img, (224, 224))
        #img_array = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0

        interpreter.set_tensor(input_details[0]['index'], [img_array])
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        # Getting corners around the face
        #faces = faceCascade.detectMultiScale(imgGray, 1.3, 5)  # 1.3 = scale factor, 5 = minimum neighbor
        # drawing bounding box around face
        
        #count += 1

        label = labels[np.argmax(output_data[0])]
        
        #put text as output_data
        cv2.putText(img, label, (20, 20), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        #cv2.putText(img, ": " + str(count), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow('face_detect', img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyWindow('face_detect')


In [34]:
predict_video(cap, interpreter, input_details, output_details, labels)

In [41]:

#  test the inference time
import time
start_time = time.time()
interpreter.invoke()
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.5196387767791748 seconds ---
